In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import roc_auc_score

In [2]:

data = pd.read_csv('E:/WPPhotometer/RZ10X/High/HighWSLS_40ex.csv')

removeBlocks = data[data['Trial#'] > 20]
removeLastTrial = removeBlocks[~removeBlocks['NextLever'].isin(['End'])]
filteredData = removeLastTrial

filteredData['NextLever'] = filteredData['NextLever'].map({'Small': 0, 'Omission': 0, 'Punished': 1, 'UnPunished':1})
filteredData['Lever'] = filteredData['Lever'].map({'Small': 0, 'Punished': 1, 'UnPunished':1})

# Step 2: Display the first few rows of the DataFrame to ensure it loaded correctly
print(filteredData.head())

# Step 3: Extract the features and target
X = filteredData[['ID', 'LeverAUC']]  # Replace 'Group ID' and 'Predictor' with your actual column names
y = filteredData['NextLever']  # Replace 'Response' with your actual response column name


    ID    Date  Trial#  Lever  NextLever     WSLS  LeverZmax  LeverZmin  \
20  16  230123      21      1          1  WinStay   2.420976  -0.977182   
21  16  230123      22      1          1  WinStay   2.825333  -1.335497   
22  16  230123      23      1          1  WinStay   1.845744  -1.398745   
23  16  230123      24      1          1  WinStay   1.844687  -1.285533   
24  16  230123      25      1          1  WinStay   0.791917  -1.576015   

     LeverAUC    HLZmax    HLZmin       HLauc  
20  83.154362  2.733061 -3.425178   18.250144  
21  48.212738  1.237365 -1.231867   -3.736264  
22  23.342977  2.711890 -0.880507   84.757391  
23 -17.232411  3.012229 -1.134604  117.119915  
24 -63.758027  0.235207 -2.486015  -72.052411  


C:\Users\wonnp\AppData\Local\Temp\ipykernel_33140\3022793693.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filteredData['NextLever'] = filteredData['NextLever'].map({'Small': 0, 'Omission': 0, 'Punished': 1, 'UnPunished':1})
C:\Users\wonnp\AppData\Local\Temp\ipykernel_33140\3022793693.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filteredData['Lever'] = filteredData['Lever'].map({'Small': 0, 'Punished': 1, 'UnPunished':1})


In [3]:

# One-hot encode the GroupID
encoder = OneHotEncoder(sparse_output=False)
group_encoded = encoder.fit_transform(filteredData[['ID']])
group_encoded_df = pd.DataFrame(group_encoded, columns=encoder.get_feature_names_out(['ID']))

# Combine the encoded group features with the original data
filteredData = pd.concat([filteredData, group_encoded_df], axis=1)
filteredData.drop(['ID'], axis=1, inplace=True)


# Standardize the continuous predictors
scaler = StandardScaler()
X[['LeverAUC']] = scaler.fit_transform(X[['LeverAUC']])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define and train the logistic regression model
model = LogisticRegression(solver='lbfgs', max_iter=1000)
model.fit(X_train, y_train)

# Make predictions
y_pred_prob = model.predict_proba(X_test)[:, 1]
y_pred = model.predict(X_test)

# Evaluate the model
auc = roc_auc_score(y_test, y_pred_prob)
print(f'Test AUC: {auc}')

# Display the coefficients
coef = model.coef_[0]
features = X.columns

print("Coefficients:")
for feature, coefficient in zip(features, coef):
    print(f'{feature}: {coefficient:.4f}')

Test AUC: 0.6006134969325153
Coefficients:
ID: -0.0068
LeverAUC: 0.1454


C:\Users\wonnp\AppData\Local\Temp\ipykernel_33140\2799323132.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[['LeverAUC']] = scaler.fit_transform(X[['LeverAUC']])
